# PDFs download

In [3]:
# use paperswithcode api
!pip install paperswithcode-client

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 KB 5.9 MB/s eta 0:00:00
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 5.4

In [3]:
import requests
import time
import json

TASK_SLUG = "knowledge-graph-embeddings"
PAPERS_URL = f"https://paperswithcode.com/api/v1/tasks/{TASK_SLUG}/papers/"

# Fetch papers
response = requests.get(PAPERS_URL)
papers_data = response.json()

information = ['tasks', 'datasets','implementations']
# Display the first few papers
papers_list= []

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching {url}: {response.status_code}")
        return None


for paper in papers_data["results"][:40]:  # Limit output to 5 papers
    paper_entry = {
        "Title": paper["title"],
        "Authors": ", ".join(paper["authors"]),
        "Abstract": paper.get("abstract", "No abstract available"),
        "PDF URL": paper["url_pdf"],
        "Datasets": [],
        "Tasks": [],
    }
    paper_slug = paper["id"]	

    dataset_url = f"https://paperswithcode.com/api/v1/papers/{paper_slug}/datasets/"
    dataset_data = fetch_data(dataset_url)
    if dataset_data:
        for dataset in dataset_data["results"]:
            paper_entry["Datasets"].append({dataset["name"]})
    
    taks_url = f"https://paperswithcode.com/api/v1/papers/{paper_slug}/tasks/"
    task_data = fetch_data(taks_url)
    if task_data:
        for task in task_data["results"]:
            paper_entry["Tasks"].append({task["name"]})
    # Append paper entry to the list
    papers_list.append(paper_entry)

    # Sleep to prevent rate limits
    time.sleep(1)



KeyboardInterrupt: 

In [2]:
import json

# Convert sets to lists before saving
def convert_sets_to_lists(obj):
    if isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, list):
        return [convert_sets_to_lists(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_sets_to_lists(value) for key, value in obj.items()}
    return obj  # Return original object if no conversion needed

# Ensure `papers_list` is fully JSON serializable
papers_list = convert_sets_to_lists(papers_list)

# Save JSON
with open("papers_data.json", "w", encoding="utf-8") as f:
    json.dump(papers_list, f, indent=4)

print("Successfully saved data in 'papers_data.json'")


Successfully saved data in 'papers_data.json'


# Download pdfs

In [25]:
import os


# Load previously saved JSON file with paper metadata
with open("papers_data.json", "r", encoding="utf-8") as f:
    papers_list = json.load(f)

# Create a folder to save PDFs
PDF_FOLDER = "papers_pdfs"
os.makedirs(PDF_FOLDER, exist_ok=True)

# Function to download a PDF
def download_pdf(pdf_url, title):
    filename = f"{PDF_FOLDER}/{title.replace(' ', '_')}.pdf"


    try:
        response = requests.get(pdf_url, stream=True)
        response.raise_for_status()  # Raise an error if the request fails
        
        with open(filename, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"Downloaded: {filename}")
        return filename
    
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {title}: {e}")
        return None
# Loop through papers and download PDFs
for paper in papers_list:
    pdf_url = paper.get("PDF URL")
    title = paper["Title"]

    if pdf_url and pdf_url.startswith("http"):
        pdf_path = download_pdf(pdf_url, title)
        if pdf_path:
            paper["Local PDF Path"] = str(pdf_path)  # Store local path in JSON
    else:
        print(f"No valid PDF URL for: {title}")
        paper["Local PDF Path"] = None  # Store None if no valid PDF

# Save the updated JSON with local paths
with open("papers_data.json", "w", encoding="utf-8") as f:
    json.dump(papers_list, f, indent=4)

print("All available PDFs have been downloaded!")


Downloaded: papers_pdfs/KG^2:_Learning_to_Reason_Science_Exam_Questions_with_Contextual_Knowledge_Graph_Embeddings.pdf
Downloaded: papers_pdfs/Incorporating_Literals_into_Knowledge_Graph_Embeddings.pdf
Downloaded: papers_pdfs/Adversarial_Contrastive_Estimation.pdf
Downloaded: papers_pdfs/KBGAN:_Adversarial_Learning_for_Knowledge_Graph_Embeddings.pdf
Downloaded: papers_pdfs/Convolutional_2D_Knowledge_Graph_Embeddings.pdf
Downloaded: papers_pdfs/Answering_Visual-Relational_Queries_in_Web-Extracted_Knowledge_Graphs.pdf
Downloaded: papers_pdfs/Expeditious_Generation_of_Knowledge_Graph_Embeddings.pdf
Downloaded: papers_pdfs/Learning_Knowledge_Graph_Embeddings_with_Type_Regularizer.pdf
Downloaded: papers_pdfs/Analysis_of_the_Impact_of_Negative_Sampling_on_Link_Prediction_in_Knowledge_Graphs.pdf
Downloaded: papers_pdfs/DeepPath:_A_Reinforcement_Learning_Method_for_Knowledge_Graph_Reasoning.pdf
Downloaded: papers_pdfs/Inducing_Interpretability_in_Knowledge_Graph_Embeddings.pdf
Downloaded: pape

## Test on the pdfs

In [14]:
import json
with open("papers_data.json", "r", encoding="utf-8") as f:
    papers_list = json.load(f)

In [3]:
def chunk_text(text, tokenizer, max_tokens=8000, overlap=200):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk = tokenizer.decode(tokens[start:end])
        chunks.append(chunk)
        start += max_tokens - overlap  # Overlapping context
    return chunks

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1: Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

max_context_tokens = 8192 - 1024

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
import pdfplumber
from pathlib import Path
import os

# Get the current working directory
current_dir = Path(os.getcwd())
parent_dir = current_dir.parent

paper = papers_list[0]


with pdfplumber.open(str(current_dir /paper['Local PDF Path'])) as pdf:
    text = "\n".join(page.extract_text() for page in pdf.pages)
# print(text)

In [6]:
question = "What are the name of datasets used in the paper?"
question2 = "What are the tasks that the model is trained for?"
question3 = "Who are the authors of the paper?"
questions = [question, question2, question3]
responses_dataset = []
responses_task = []
responses_authors = []

In [7]:
for paper in papers_list[0:20]:
    with pdfplumber.open(str(current_dir /paper['Local PDF Path'])) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages)
        
    # Example Usage
    chunks = chunk_text(text, tokenizer, max_tokens=max_context_tokens, overlap=200)
    
    # **Add context chunks iteratively**
    for i, question in enumerate(questions):
        response= []
        for j, chunk in enumerate(chunks):

            chat = [
                {"role": "system", "content": "You are an assistant for question-answering tasks. Use only the provided context information to form your response."}
            ]


            chat.append({"role": "user", "content": f"Context chunk: {chunk}"})

            chat.append({"role": "user", "content": f"Now, given this question: {question}.  Give back the answer only in a Python list format, for example: ['A','B']. If you don't know the answer, just return an empty list."})

            # chat = [
            #     {"role": "system", "content":"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question later on. If you don't know the answer, just say that you don't know." },
            #     {"role": "user", "content": f"Question: {question} Context: {context}"}
            # ]

            # 2: Apply the chat template
            formatted_chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
            print("Formatted chat:\n", formatted_chat)

            # 3: Tokenize the chat (This can be combined with the previous step using tokenize=True)
            inputs = tokenizer(formatted_chat, return_tensors="pt", add_special_tokens=False)
            # Move the tokenized inputs to the same device the model is on (GPU/CPU)
            inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}
            print("Tokenized inputs:\n", inputs)

            # 4: Generate text from the model
            outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.1)
            print("Generated tokens:\n", outputs)

            # 5: Decode the output back to a string
            decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
            print("Decoded output:\n", decoded_output)

            response.append(decoded_output)
        if i == 0:
            responses_dataset.append(response)
        elif i == 1:
            responses_task.append(response)
        else:
            responses_authors.append(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Formatted chat:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for question-answering tasks. Use only the provided context information to form your response.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context chunk: KG2: Learning to Reason Science Exam Questions
with Contextual Knowledge Graph Embeddings
Yuyu Zhang1∗, Hanjun Dai1∗, Kamil Toraman2, and Le Song1
1College of Computing, Georgia Institute of Technology
2Korea Advanced Institute of Science and Technology
1{yuyu.zhang, hanjun.dai, lsong}@cc.gatech.edu
2kvtoraman@kaist.ac.kr
Abstract
TheAI2ReasoningChallenge(ARC),anewbenchmarkdatasetforquestionanswering(QA)
has been recently released. ARC only contains natural science questions authored for human
exams, which are hard to answer and require advanced logic reasoning. On the ARC Chal-
lengeSet, existingstate-of-the-artQAsystemsfailtosignificantlyoutperformrandombaseline,
reflecting the difficult nature of this task. In this paper, we pr

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated tokens:
 tensor([[128000, 128006,   9125,  ...,  45895,    663, 128009]],
       device='cuda:0')
Decoded output:
 ['ARC', 'SQuAD', 'SNLI', 'SciTail']
Formatted chat:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for question-answering tasks. Use only the provided context information to form your response.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context chunk: KG2: Learning to Reason Science Exam Questions
with Contextual Knowledge Graph Embeddings
Yuyu Zhang1∗, Hanjun Dai1∗, Kamil Toraman2, and Le Song1
1College of Computing, Georgia Institute of Technology
2Korea Advanced Institute of Science and Technology
1{yuyu.zhang, hanjun.dai, lsong}@cc.gatech.edu
2kvtoraman@kaist.ac.kr
Abstract
TheAI2ReasoningChallenge(ARC),anewbenchmarkdatasetforquestionanswering(QA)
has been recently released. ARC only contains natural science questions authored for human
exams, which are hard to answer and require advanced logic reasoning. On the ARC

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Generated tokens:
 tensor([[128000, 128006,   9125,  ...,    287,    663, 128009]],
       device='cuda:0')
Decoded output:
 ['Multi-Hop Knowledge Graph Reasoning', 'Query Answering']
Formatted chat:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for question-answering tasks. Use only the provided context information to form your response.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context chunk: tinueimprovingtheperformancebuthadtostopat512due
tomemoryconstraints. 5Wereportthemodelrobustnessmeasurementsin§A.1.
Model UMLSKinshipFB15k237WN18RRNELL995 995 does not change the results significantly. In
general,removingactiondropouthasagreaterim-
Ours(ConvE) 73.0 75.0 38.2 43.8 78.8
−RS 67.7 66.5 35.1 45.7 78.4 pact, suggesting that thorough exploration of the
−AD 61.3 65.4 31.0 39.1 76.1 pathspaceisimportantacrossdatasets.
Table3: ComparisonofdevsetMRRofOurs(ConvE)
5.3 Analysis
andmodelswithoutrewardshapingandactiondropout.
5.3.1 ConvergenceRate


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated tokens:
 tensor([[128000, 128006,   9125,  ...,  21579,    663, 128009]],
       device='cuda:0')
Decoded output:
 ['Knowledge Graph Query Answering', 'Multi-Hop Reasoning', 'End-to-End Reinforcement Learning']
Formatted chat:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for question-answering tasks. Use only the provided context information to form your response.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context chunk: Multi-Hop Knowledge Graph Reasoning with Reward Shaping
XiVictoriaLin RichardSocher CaimingXiong
SalesforceResearch
{xilin,rsocher,cxiong}@salesforce.com
Abstract belong_to?
U.S. Government Rudy_Giulian
belong_to belong_to collaborate
Multi-hop reasoning is an effective approach _with
collaborate_with
for query answering (QA) over incomplete Barack_Obama John_McCain
knowledgegraphs(KGs). Theproblemcanbe
endorsed_by
formulated in a reinforcement learning (RL) born_in collaborate_with? live_in col _la wb io thrate
s

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated tokens:
 tensor([[128000, 128006,   9125,  ...,     70,    663, 128009]],
       device='cuda:0')
Decoded output:
 ['Xi Victoria Lin', 'Richard Socher', 'Caiming Xiong']
Formatted chat:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for question-answering tasks. Use only the provided context information to form your response.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context chunk: tinueimprovingtheperformancebuthadtostopat512due
tomemoryconstraints. 5Wereportthemodelrobustnessmeasurementsin§A.1.
Model UMLSKinshipFB15k237WN18RRNELL995 995 does not change the results significantly. In
general,removingactiondropouthasagreaterim-
Ours(ConvE) 73.0 75.0 38.2 43.8 78.8
−RS 67.7 66.5 35.1 45.7 78.4 pact, suggesting that thorough exploration of the
−AD 61.3 65.4 31.0 39.1 76.1 pathspaceisimportantacrossdatasets.
Table3: ComparisonofdevsetMRRofOurs(ConvE)
5.3 Analysis
andmodelswithoutrewardshapingandactiondropout.
5.3.1 ConvergenceRate
out 

In [ ]:
predictions = [responses_authors, responses_dataset, responses_task]

In [ ]:
import json

data = {
    "E_pred": predictions
}

with open("qa_entities.json", "w") as f:
    json.dump(data, f, indent=2)

## Evaluation


In [18]:
import json
with open("qa_entities.json", "r") as f:
    data = json.load(f)
    predictions = data["E_pred"]

In [19]:
import ast
from rapidfuzz import fuzz, process
from itertools import chain
def deduplicate_fuzzy(names, threshold=80):
    unique = []
    for name in names:
        if all(fuzz.ratio(name, existing) < threshold for existing in unique):
            unique.append(name)
    return unique

In [20]:
responses_dataset_copy = predictions[1]

In [21]:
for i in range(len(responses_dataset_copy)):
    for j in range(len(responses_dataset_copy[i])):
        if isinstance(responses_dataset_copy[i][j], str):
            responses_dataset_copy[i][j] = ast.literal_eval(responses_dataset_copy[i][j])
    if len(responses_dataset_copy[i]) > 1:
        merged= list(chain.from_iterable(responses_dataset_copy[i]))
        responses_dataset_copy[i] = deduplicate_fuzzy(merged, threshold=80)
    else:
        responses_dataset_copy[i] = deduplicate_fuzzy(responses_dataset_copy[i][j], threshold=80)


In [ ]:
responses_dataset_copy

In [23]:
responses_task_copy = predictions[2]

In [24]:
for i in range(len(responses_task_copy)):
    for j in range(len(responses_task_copy[i])):
        if isinstance(responses_task_copy[i][j], str):

            responses_task_copy[i][j] = ast.literal_eval(responses_task_copy[i][j])
    if len(responses_task_copy[i]) > 1:
        merged= list(chain.from_iterable(responses_task_copy[i]))
        responses_task_copy[i] = deduplicate_fuzzy(merged, threshold=80)
    else:
        responses_task_copy[i] = deduplicate_fuzzy(responses_task_copy[i][j], threshold=80)

In [ ]:
responses_task_copy

In [ ]:
responses_authors_copy = predictions[0]
# return the first response, because the authors are in the first chunk always
for i, response in enumerate(responses_authors_copy):
        responses_authors_copy[i] = response[0]

responses_authors_copy

["['Yuyu Zhang', 'Hanjun Dai', 'Kamil Toraman', 'Le Song']",
 "['Agustinus Kristiadi', 'Mohammad Asif Khan', 'Denis Lukovnikov', 'Jens Lehmann', 'Asja Fischer']",
 "['AvishekJoeyBose', 'HuanLing', 'YanshuaiCao']",
 "['Liwei Cai', 'William Yang Wang']",
 "['Tim Dettmers', 'Pasquale Minervini', 'Pontus Stenetorp', 'Sebastian Riedel']",
 "['Daniel Oñoro-Rubio', 'Mathias Niepert', 'Alberto García-Durán', 'Roberto González-Sánchez', 'Roberto J. López-Sastre']",
 "['Tommaso Soru', 'Stefano Ruberto', 'Diego Moussallem', 'André Valdestilhas', 'Alexander Bigerl', 'Edgard Marx', 'Diego Esteves']",
 "['Bhushan Kotnis', 'Vivi Nastase']",
 "['Bhushan Kotnis', 'Vivi Nastase']",
 "['Wenhan Xiong', 'Thien Hoang', 'William Yang Wang']",
 "['Chandrahas', 'Tathagata Sengupta', 'Cibi Pragadeesh', 'Partha Pratim Talukdar']",
 "['Armand Joulin', 'Edouard Grave', 'Piotr Bojanowski', 'Tomas Mikolov', 'Ajoulin@fb.com', 'Egrave@fb.com', 'Bojanowski@fb.com', 'Maxn@fb.com', 'Tmikolov@fb.com']",
 "['Théo Trouillon

In [ ]:
# for i in range(len(responses_authors_copy)):

#     responses_authors_copy[i] = ast.literal_eval(responses_authors_copy[i])

In [15]:
papers = papers_list[0:20]
authors = []
datasets = []
tasks = []

# get the authors from papers
for i, paper in enumerate(papers):
    authors.append(paper['Authors'])
    datasets.append(paper['Datasets'])
    tasks.append(paper['Tasks'])

In [16]:
references = [authors, datasets, tasks]
predictions = [responses_authors_copy, responses_dataset_copy, responses_task_copy]

In [45]:
from sentence_transformers import SentenceTransformer, util
# Load sentence-level embedding model
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
autor_recalls = []
dataset_recalls = []
task_recalls = []
for i in range(len(predictions)):
    for j in range(len(predictions[i])):
        reference = references[i][j]
        prediction = predictions[i][j]

        if i != 0:
            # Flatten reference list into a single string (first elements or all)
            ref_text = [item[0] for item in reference]
            pred_text = prediction
        else:
            ref_text = [name.strip() for name in reference.split(',')]
            pred_text = ast.literal_eval(prediction)
        if len(ref_text) == 0:
            recall = 1
        elif len(pred_text) == 0:
            recall = 0
        else:
            # Compute embeddings
            ref_emb = model.encode(ref_text, convert_to_tensor=True)
            pred_emb = model.encode(pred_text, convert_to_tensor=True)

            # Compute cosine similarities: shape (len(ref), len(pred))
            cos_sim = util.pytorch_cos_sim(ref_emb, pred_emb)

            # For each reference entity, find the max similarity to predicted entities
            max_similarities = cos_sim.max(dim=1).values

            # Apply threshold
            threshold = 0.7
            num_matched = (max_similarities >= threshold).sum().item()
            recall = num_matched / len(ref_text) 
        if i == 0:
            autor_recalls.append(recall)
        elif i == 1:
            dataset_recalls.append(recall)
        elif i == 2:
            task_recalls.append(recall)
        print(f"Reference: {ref_text}")
        print(f"Prediction: {pred_text}")
        print(f"Recall: {recall:.4f}")

Reference: ['Hanjun Dai', 'Le Song', 'Kamil Toraman', 'Yuyu Zhang']
Prediction: ['Yuyu Zhang', 'Hanjun Dai', 'Kamil Toraman', 'Le Song']
Recall: 1.0000
Reference: ['Jens Lehmann', 'Mohammad Asif Khan', 'Denis Lukovnikov', 'Asja Fischer', 'Agustinus Kristiadi']
Prediction: ['Agustinus Kristiadi', 'Mohammad Asif Khan', 'Denis Lukovnikov', 'Jens Lehmann', 'Asja Fischer']
Recall: 1.0000
Reference: ['Yanshuai Cao', 'Avishek Joey Bose', 'Huan Ling']
Prediction: ['AvishekJoeyBose', 'HuanLing', 'YanshuaiCao']
Recall: 0.0000
Reference: ['William Yang Wang', 'Liwei Cai']
Prediction: ['Liwei Cai', 'William Yang Wang']
Recall: 1.0000
Reference: ['Pontus Stenetorp', 'Sebastian Riedel', 'Tim Dettmers', 'Pasquale Minervini']
Prediction: ['Tim Dettmers', 'Pasquale Minervini', 'Pontus Stenetorp', 'Sebastian Riedel']
Recall: 1.0000
Reference: ['Roberto J. López-Sastre', 'Roberto González', 'Alberto García-Durán', 'Daniel Oñoro-Rubio', 'Mathias Niepert']
Prediction: ['Daniel Oñoro-Rubio', 'Mathias Nieper

In [39]:
mean_autor_recall = sum(autor_recalls) / len(autor_recalls)
mean_dataset_recall = sum(dataset_recalls) / len(dataset_recalls)
mean_task_recall = sum(task_recalls) / len(task_recalls)
print(f"Mean Author Recall: {mean_autor_recall:.4f}")
print(f"Mean Dataset Recall: {mean_dataset_recall:.4f}")
print(f"Mean Task Recall: {mean_task_recall:.4f}")

Mean Author Recall: 0.8900
Mean Dataset Recall: 0.8554
Mean Task Recall: 0.2328
